### 2016-01 Recuperatorio

Una red social almacena el contenido de los chats entre sus usuarios
en un RDD que tiene registros con el siguiente formato: (chat_id,
user_id, nickname, text). Queremos saber cuál es el usuario (user_id)
que mas preguntas hace en sus chats, contabilizamos una pregunta por
cada caracter “?” que aparezca en el campo text. Programar en Spark
un programa que identifique al usuario preguntón.

In [5]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!???'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias????'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [6]:
data = sc.parallelize(chats);

In [8]:
data.collect()

[(1, 1, 'damu', 'Qu\xc3\xa9 es esto?'),
 (2, 2, 'martin', 'Un chat!???'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'S\xc3\xad! C\xc3\xb3mo sabias????'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [7]:
data.map(lambda x: (x[1], x[3].count('?')))\
    .reduceByKey(lambda x,y: x+y)\
    .reduce(lambda x,y: x if x[1]>y[1] else y)

(2, 7)

### 2016-01 Parcial

UBER almacena en un cluster todos los datos sobre el movimiento y
viajes de todos sus vehículos. Existe un proceso que nos devuelve un
RDD llamado trip_summary con los siguientes campos: (driver_id,
car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled),
Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para
Abril de 2016.

In [9]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [10]:
data = sc.parallelize(trips);

In [11]:
data.collect()

[(1, 1, 1, 1, '20160101', 10),
 (2, 2, 2, 2, '20160202', 20),
 (1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40),
 (3, 3, 8, 2, '20160505', 80)]

In [12]:
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500')\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .reduce(lambda x,y: x if float(x[1][1])/x[1][0] > float(y[1][1])/y[1][0] else y)
         

(2, (2, 65))

### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [13]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [14]:
data = sc.parallelize(stars);

In [15]:
data.collect()

[(1, 5, 1, 1),
 (2, 10, 1, 1),
 (3, 6, 1, 1),
 (4, 5.5, 2, 1),
 (1, 6, 1, 2),
 (2, 9, 1, 2),
 (3, 5, 1, 2),
 (1, 5.5, 1, 3),
 (2, 11, 1, 3),
 (3, 5.5, 1, 3)]

In [20]:
data.map(lambda x: (x[0], (x[1], x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1],x[2]+y[2]))\
    .map(lambda x: ((x[1][1],round(float(x[1][0])/x[1][2],1)),x[0]))\
    .groupByKey()\
    .filter(lambda x: len(x[1]) > 1)\
    .map(lambda x: (list(x[1])))\
    .collect()

[[1, 3]]

### 2016-02 Parcial

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [21]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [22]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [24]:
words = documentsRDD.flatMap(lambda x: [(word, x[0]) for word in x[1].split()])
words.collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [25]:
joinWordsScores = scoresRDD.map(lambda x: (x[1],(x[0],x[2])))\
                .join(words)

In [26]:
joinWordsScores.collect()

[('honey', (('thom', 1), 1)),
 ('honey', (('martin', -1), 1)),
 ('ok', (('martin', 30), 3)),
 ('pablo', (('thom', 1), 1)),
 ('pablo', (('martin', -1), 1)),
 ('computer', (('martin', 30), 3))]

In [28]:
joinWordsScores.map(lambda x: ((x[1][0][0],x[1][1]),x[1][0][1]))\
               .reduceByKey(lambda x,y: x+y)\
               .collect()

[(('martin', 3), 60), (('thom', 1), 2), (('martin', 1), -2)]

### 2017-01 Parcial

Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

In [1]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

players_all_time_stats_rdd = sc.parallelize(players_all_time_stats)
scores_rdd = sc.parallelize(scores)

In [2]:
# cambio de clave a id_jugador, partido
scores_by_match = scores_rdd.map(lambda a: ((a[0],a[1]),a[3]))
scores_by_match = scores_by_match.reduceByKey(lambda a,b: a+b)
scores_by_match.collect()

[((1, 1), 200), ((1, 2), 300), ((3, 2), 100), ((2, 1), 300)]

In [3]:
# cambio de clave a id_jugador
scores_by_player = scores_by_match.map(lambda a: (a[0][0], a[1]))
scores_by_player.collect()

[(1, 200), (1, 300), (3, 100), (2, 300)]

In [4]:
# juntamos los datos para poder evaluar lo pedido
# preparamos la key del lado de informacion historica de jugadores.

all_time_with_key = players_all_time_stats_rdd.map(lambda a: (a[0],a))
all_time_with_key.collect()

[(1, (1, 'Manu Ginobili', 800)),
 (2, (2, 'Kobe Bryant', 100)),
 (3, (3, 'Marc Gasol', 25)),
 (4, (4, 'James Harden', 1000))]

In [5]:
scores_by_player = scores_by_player.join(all_time_with_key)
scores_by_player.collect()

[(1, (200, (1, 'Manu Ginobili', 800))),
 (1, (300, (1, 'Manu Ginobili', 800))),
 (2, (300, (2, 'Kobe Bryant', 100))),
 (3, (100, (3, 'Marc Gasol', 25)))]

In [6]:
overperformers = scores_by_player.filter(lambda a: a[1][0] > a[1][1][2])
overperformers.collect()

[(2, (300, (2, 'Kobe Bryant', 100))), (3, (100, (3, 'Marc Gasol', 25)))]

In [7]:
# veamos los duplicados
overperformers = overperformers.map(lambda a: a[1][1][1]).distinct()
overperformers.collect()

['Kobe Bryant', 'Marc Gasol']